In [65]:
import os
import ctypes
__here__ = os.path.abspath(".")

import pygccxml

from pygccxml import utils
from pygccxml import declarations
from pygccxml import parser
import os
import subprocess

# Configure the xml generator
try:
    gcc_path = subprocess.check_output(["where", "gcc"]).decode("UTF-8").strip()
    gcc_path = os.path.abspath(gcc_path)
    assert os.path.isfile(gcc_path)
except subprocess.CalledProcessError:
    pass

os.chdir(r"castxml\bin")
generator_path, generator_name = utils.find_xml_generator()
os.chdir(__here__)

xml_generator_config = parser.xml_generator_configuration_t(
    xml_generator_path=generator_path,
    xml_generator=generator_name,
    compiler_path=gcc_path
)

In [67]:
header = os.path.abspath("Development/CANapAPI.h")

os.path.exists(header)

header_declarations = parser.parse(
    files=[header],
    config=xml_generator_config
)

INFO Parsing source file "C:\Projects\python_CANape\Development\CANapAPI.h" ... 


RuntimeError: Error occurred while running CASTXML:  status:1

fatal error: too many errors emitted, stopping now [-ferror-limit=]
2 warnings and 20 errors generated.

In [68]:
with open(header, "r") as fid:
    data = fid.read()

In [69]:
include_fix = """
#include <windows.h>
#include <iostream.h>
#include <stdio.h>
#include <conio.h>
#include <time.h>
#include <dos.h>
#include <limits.h>
#include <float.h>
"""

In [70]:
header_fix = """
#define _MAX_PATH 256
#define MAX_PATH 256
#define CALLBACK __stdcall
typedef int BOOL;
typedef unsigned long DWORD;
typedef unsigned char BYTE;
typedef BYTE BOOLEAN;
typedef unsigned int UINT;
typedef unsigned long ULONG_PTR;
typedef ULONG_PTR DWORD_PTR;
#define NULL 0
"""

In [71]:
os.path.splitext(header)

('C:\\Projects\\python_CANape\\Development\\CANapAPI', '.h')

In [72]:
base, ext = os.path.splitext(header)
header2 = os.path.abspath(f"{base}2{ext}")

In [73]:
fid = open(header2, "w")

In [74]:
fid.write(include_fix)

156

In [75]:
fid.write(data)

140766

In [76]:
fid.close()

In [78]:
header_declarations = parser.parse(
    files=[header2],
    config=xml_generator_config
)

INFO Parsing source file "C:\Projects\python_CANape\Development\CANapAPI2.h" ... 


RuntimeError: Error occurred while running CASTXML:  status:1

In [79]:
with open(header2, "w") as fid:
    fid.write(header_fix)
    fid.write(data)
header_declarations = parser.parse(
    files=[header2],
    config=xml_generator_config
)

INFO Parsing source file "C:\Projects\python_CANape\Development\CANapAPI2.h" ... 


RuntimeError: Error occurred while running CASTXML:  status:1

In [82]:
header_fix = """
// Push these out of the way
#define ASAP3_EXPORT
#define CALL_CONV
#define WINAPI

// Windows Path
#define _MAX_PATH 256
#define MAX_PATH 256
#define CALLBACK __stdcall

// Some typedefs
typedef int BOOL;
typedef unsigned long DWORD;
typedef unsigned char BYTE;
typedef BYTE BOOLEAN;
typedef unsigned int UINT;
typedef unsigned long ULONG_PTR;
typedef ULONG_PTR DWORD_PTR;
#define NULL 0
"""

with open(header2, "w") as fid:
    fid.write(header_fix)
    fid.write(data)
header_declarations = parser.parse(
    files=[header2],
    config=xml_generator_config
)

INFO Parsing source file "C:\Projects\python_CANape\Development\CANapAPI2.h" ... 


In [86]:
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    print(typedef)
    break

BOOL [typedef]


In [95]:
typedef.decl_string.lstrip()

'::BOOL'

In [88]:
typedef.decl_type

In [89]:
typedef.decl_type.decl_string

'int'

In [91]:
hasattr(ctypes, "c_int")

True

In [94]:
if hasattr(ctypes, f"c_{typedef.decl_type.decl_string}"):
    print(f"c_{typedef.decl_type.decl_string}")

c_int


In [99]:
type_name = typedef.decl_string.lstrip(":")

In [101]:
print(f"{type_name}=ctypes.c_{typedef.decl_type.decl_string}")

BOOL=ctypes.c_int


In [102]:
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    if hasattr(ctypes, f"c_{typedef.decl_type.decl_string}"):
        print(f"{type_name}=ctypes.c_{typedef.decl_type.decl_string}")
        continue
    break

BOOL=ctypes.c_int


In [103]:
typedef.decl_string.lstrip(), typedef.decl_type, typedef.decl_type.decl_string

('::DWORD',
 'long unsigned int')

In [109]:
class ctypes_typedef_gen(object):
    def __init__(self, typedef):
        self.typedef=typedef
    
    @property
    def decl_str(self):
        return self.typedef.decl_type.decl_string
    
    @property
    def unsigned(self):
        return "unsigned" in self.decl_str
        
    @property
    def name(self):
        return self.typedef.name
    
    @property
    def short_name(self):
        return self.name.split("_")[0]
    
    @property
    def c_type(self):
        if hasattr(ctypes, f"c_{self.short_name}"):
            return f"c_{self.short_name}"
        if hasattr(ctypes, f"c_{self.decl_str}"):
            return f"c_{self.decl_str}"
        type_map = {
            "unsigned char": "c_ubyte",
            "void *": "c_void_p",
            "long unsigned int": "c_ulong",
        }
        if self.decl_str in type_map:
            return type_map[self.decl_str]
        
        raise Exception(f"Unimplemented type {self.decl_str}")
        
    @property
    def python_line(self):
        return f"{self.name}=ctypes.{self.c_type}"
    def __repr__(self):
        return "<{}={}>".format(self.name, self.decl_str)

In [110]:
n = ctypes_typedef_gen(typedef)

In [111]:
n.c_type

'c_ulong'

In [116]:
deffed = list()
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    if typedef.name in deffed:
        print(f"{typedef.name}={deff})
        
    n = ctypes_typedef_gen(typedef)
    print(f"{n.name}={n.c_type}")
    deffed.append(n.name)
    continue
    if hasattr(ctypes, f"c_{typedef.decl_type.decl_string}"):
        print(f"{type_name}=ctypes.c_{typedef.decl_type.decl_string}")
        continue
    break

BOOL=c_int
DWORD=c_ulong
BYTE=c_ubyte


Exception: Unimplemented type ::BYTE

In [117]:
typedef.decl_string.lstrip(), typedef.decl_type, typedef.decl_type.decl_string

('::BOOLEAN',
 '::BYTE')

In [84]:
for decl in ns.declarations:
    if decl.name=="TAsap3Hdl":
        print(decl)
        break

TAsap3Hdl [typedef]


In [51]:
for decl in ns.declarations:
    if decl.name=="tAsap3Hdl":
        print(decl)
        break

tAsap3Hdl [class declaration]


In [52]:
decl.decl_string

'::tAsap3Hdl'

In [56]:
decl.

False

In [27]:
typedef.decl_type.decl_string

'long int (*)( ::TAsap3Hdl,long unsigned int )'

In [28]:
subset1=list()
subset2=list()
for typedef in ns.typedefs():
    if typedef.name.startswith("_"):
        continue
    if typedef.decl_type.decl_string.startswith("::"):
        subset1.append(typedef)
    else:
        subset2.append(typedef)

class ctypes_typedef_gen(object):
    def __init__(self, typedef):
        self.typedef=typedef
    
    @property
    def decl_str(self):
        return self.typedef.decl_type.decl_string
    
    @property
    def unsigned(self):
        return "unsigned" in self.decl_str
        
    @property
    def name(self):
        return self.typedef.name
    
    @property
    def short_name(self):
        return self.name.split("_")[0]
    
    @property
    def c_type(self):
        if hasattr(ctypes, f"c_{self.short_name}"):
            return f"c_{self.short_name}"
        if hasattr(ctypes, f"c_{self.decl_str}"):
            return f"c_{self.decl_str}"
        type_map = {
            "unsigned char": "c_ubyte",
            "void *": "c_void_p",
        }
        if self.decl_str in type_map:
            return type_map[self.decl_str]
        
        raise Exception(f"Unimplemented type {self.decl_str}")
        
    @property
    def python_line(self):
        return f"{self.name}=ctypes.{self.c_type}"
    def __repr__(self):
        return "<{}={}>".format(self.name, self.decl_str)

In [29]:
ctypes.c_void_p

ctypes.c_void_p

In [18]:
n = [ctypes_typedef_gen(c) for c in subset2]
n

[<int8_T=signed char>,
 <uint8_T=unsigned char>,
 <int16_T=short int>,
 <uint16_T=short unsigned int>,
 <int32_T=int>,
 <uint32_T=unsigned int>,
 <int64_T=long long int>,
 <uint64_T=long long unsigned int>,
 <real32_T=float>,
 <real64_T=double>,
 <real_T=double>,
 <time_T=double>,
 <boolean_T=unsigned char>,
 <int_T=int>,
 <uint_T=unsigned int>,
 <ulong_T=long unsigned int>,
 <ulonglong_T=long long unsigned int>,
 <char_T=char>,
 <uchar_T=unsigned char>,
 <pointer_T=void *>]

In [23]:
[print(o.python_line) for o in n]

int8_T=ctypes.c_int8
uint8_T=ctypes.c_uint8
int16_T=ctypes.c_int16
uint16_T=ctypes.c_uint16
int32_T=ctypes.c_int32
uint32_T=ctypes.c_uint32
int64_T=ctypes.c_int64
uint64_T=ctypes.c_uint64
real32_T=ctypes.c_float
real64_T=ctypes.c_double
real_T=ctypes.c_double
time_T=ctypes.c_double
boolean_T=ctypes.c_ubyte
int_T=ctypes.c_int
uint_T=ctypes.c_uint
ulong_T=ctypes.c_ulong
ulonglong_T=ctypes.c_ulonglong
char_T=ctypes.c_char
uchar_T=ctypes.c_ubyte
pointer_T=ctypes.c_void_p


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [31]:
[ctypes_typedef_gen(n) for n in subset1]

[<byte_T=::char_T>,
 <creal32_T=::creal32_T>,
 <creal64_T=::creal64_T>,
 <creal_T=::creal_T>,
 <cint8_T=::cint8_T>,
 <cuint8_T=::cuint8_T>,
 <cint16_T=::cint16_T>,
 <cuint16_T=::cuint16_T>,
 <cint32_T=::cint32_T>,
 <cuint32_T=::cuint32_T>,
 <cint64_T=::cint64_T>,
 <cuint64_T=::cuint64_T>]

In [32]:
o=subset1[-1]

In [33]:
o.decl_string

'::cuint64_T'

In [34]:
o.decl_type

In [39]:
[o.decl_type.decl_string == o.decl_string for o in subset1]

[False, True, True, True, True, True, True, True, True, True, True, True]

In [45]:
o = ns.decls("::cuint64_T")

In [50]:
o.declarations

In [51]:
[q.name for q in o.declarations]

['cuint64_T', '', '', 'cuint64_T']

In [58]:
ns.decls("cuint64_T").declarations[0].declarations

In [59]:
ns.decls("::cuint64_T").declarations[0].declarations

In [60]:
q = o[1]

In [67]:
[q.name for q in o.declarations]

['cuint64_T', '', '', 'cuint64_T']

In [68]:
[q.is_artificial for q in o.declarations]

[False, True, True, False]

In [72]:
ns.decls("::uchar_T")[:]

In [73]:
# Variables that are typedefed from other variables, but aren't structs.
subset1_1 = [o for o in subset1 if not o.decl_type.decl_string == o.decl_string]

In [76]:
m = subset1_1[0]

In [78]:
m

In [97]:
from vardump import vardump

In [98]:
vardump(m)

attributes: <class 'NoneType'><None>
byte_align: <class 'float'><1.0>
byte_size: <class 'float'><1.0>
cache: <class 'pygccxml.declarations.algorithms_cache.declaration_algs_cache_t'><<pygccxml.declarations.algorithms_cache.declaration_algs_cache_t object at 0x000001AD216BD940>>
create_decl_string(): ::byte_T
decl_string: <class 'str'><::byte_T>
decl_type: <class 'pygccxml.declarations.cpptypes.declarated_t'><char_T>
decorated_name: <class 'NoneType'><None>
demangled: <class 'NoneType'><None>
get_mangled_name(): None
i_depend_on_them(): [<pygccxml.declarations.class_declaration.dependency_info_t object at 0x000001AD216D2F98>]
is_artificial: <class 'bool'><False>
location: <class 'pygccxml.declarations.location.location_t'><<pygccxml.declarations.location.location_t object at 0x000001AD216BD8D0>>
mangled: <class 'NoneType'><None>
name: <class 'str'><byte_T>
parent: <class 'pygccxml.declarations.namespace.namespace_t'><:: [namespace]>
partial_decl_string: <class 'str'><::byte_T>
partial_n

C:\Projects\bootstrap_WinPython\WinPython\python-3.6.6.amd64\lib\site-packages\pygccxml\declarations\declaration.py:303: DeprecationWarning: The decorated_name attribute is deprecated. See the changelog.
  DeprecationWarning)


'char_T'

In [ ]:
vardump()

In [ ]:
for typedef in ns.typedefs():
    print(typedef)
    if typedef.name.endswith("_T"):
        break 